Universidade Federal de Alagoas

IC - Instituto de Computação

 

# Processamento de linguagem natural - 2020.1
**Professor**: Thales Vieira

**Alunos**: Hugo Tallys Martins Oliveira e Valério Nogueira Rodrigues Júnior


## 6ª lista de exercícios

---

### Pré-processamento dos dados

In [23]:
import nltk
import numpy
import pandas
import random
import re
import time
import umap
import pickle

from bokeh.io import output_notebook, show
from bokeh.palettes import Category20
from bokeh.plotting import figure
from gensim.models import KeyedVectors
from IPython.display import HTML, display
from keras.layers import Conv1D, Dense, Embedding, LSTM
from keras.models import Sequential, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF, PCA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from yellowbrick.cluster import KElbowVisualizer

In [2]:
output_notebook() # Necessário para visualizar os gráficos com bokeh

Loading BokehJS ...

In [3]:
nltk.download('stopwords'); nltk.download('rslp'); nltk.download('punkt');

[nltk_data] Downloading package stopwords to C:\Users\Hugo -
[nltk_data]     LED\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to C:\Users\Hugo -
[nltk_data]     LED\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Hugo -
[nltk_data]     LED\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
bbc_dataset_url = 'data/bbc.csv'
cnn_dataset_url = 'data/cnn.csv'

bbc = pandas.read_csv(bbc_dataset_url, sep='|')
bbc['label'] = 'BBC'
cnn = pandas.read_csv(cnn_dataset_url, sep='|')
cnn['label'] = 'CNN'

dataset = pandas.concat([bbc, cnn], ignore_index=True)
dataset = dataset.dropna(axis=0).reset_index(drop=True)

In [5]:
dataset.text = dataset.text.apply(lambda text: text.replace('\n', ' ')) # Remoção das quebras de linha
dataset.title = dataset.title.apply(lambda text: text.replace('\n', ' '))

In [6]:
def remove_boilerplate(text):
    boilerplate = ['Compartilhe este post com Email Facebook Messenger Messenger Twitter WhatsApp LinkedIn Copiar este link Estes são links externos e abrirão numa nova janela', 'Já assistiu aos nossos novos vídeos no YouTube? Inscreva-se no nosso canal!', 'Final de YouTube post  de BBC News Brasil Final de YouTube post 2 de BBC News Brasil Final de YouTube post 3 de BBC News Brasil']
    
    for b in boilerplate:
        text = text.replace(b, '')
    return text

dataset.text = dataset.text.apply(remove_boilerplate) # Remoção de fragmentos irrelevantes do texto que se repetem

In [7]:
def preprocess(text):
    text = re.sub(r'\w*\d\w*', '', text) # Remove todas as palavras que contém números
    text = re.sub(r'[^a-zA-ZàáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', '', text.lower()) # Remove pontuação e converte para minúscula
    return re.sub(r'\s+', ' ', text) # Remove espaços repetidos

dataset['processed_text'] = dataset.text.apply(preprocess)

In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def tokenize_remove_stopwords(text, only_tokenize=False):
    tokenized_text = nltk.word_tokenize(text, language='portuguese')
    
    if only_tokenize:
        return ' '.join([token for token in tokenized_text])
    return ' '.join([token for token in tokenized_text if token not in stopwords])

dataset['processed_text_'] = dataset.processed_text.apply(lambda t: tokenize_remove_stopwords(t, True)) # Tokeniza o texto e NÃO remove stopwords
dataset.processed_text = dataset.processed_text.apply(lambda t: tokenize_remove_stopwords(t)) # Tokeniza o texto e remove stopwords

In [9]:
dataset

,url,title,text,label,processed_text,processed_text_
0,https://www.bbc.co.uk/portuguese/brasil-53020785,Coronavírus: pandemia pode jogar até 14 milhõe...,A turbulência econômica causada pela pandemi...,BBC,turbulência econômica causada pandemia novo co...,a turbulência econômica causada pela pandemia ...
1,https://www.bbc.co.uk/portuguese/brasil-53027318,Coronavírus: como funcionam as duas vacinas co...,Cerca de 11 mil voluntários brasileiros vão ...,BBC,cerca mil voluntários brasileiros vão receber ...,cerca de mil voluntários brasileiros vão receb...
2,https://www.bbc.co.uk/portuguese/brasil-51713943,Coronavírus: Brasil passa o Reino Unido e se t...,*atualizada às 18h20 de 12 de junho de 2020 ...,BBC,atualizada junho brasil totalizou nesta sextaf...,atualizada às de de junho de o brasil totalizo...
3,https://www.bbc.co.uk/portuguese/internacional...,Coronavírus na Índia: com lockdown 'insustentá...,"Quando, em 24 de março, o governo indiano in...",BBC,março governo indiano iniciou estrito isolamen...,quando em de março o governo indiano iniciou u...
4,https://www.bbc.co.uk/portuguese/internacional...,2ª onda do coronavírus? Irã vê aumento acelera...,O Irã registrou um rápido aumento no número ...,BBC,irã registrou rápido aumento número casos covi...,o irã registrou um rápido aumento no número de...
...,...,...,...,...,...,...
1611,https://www.cnnbrasil.com.br/saude/2020/02/27/...,Farmácias têm falta de máscaras após confirmaç...,Com a confirmação do primeiro caso de contamin...,CNN,confirmação primeiro caso contaminação novo co...,com a confirmação do primeiro caso de contamin...
1612,https://www.cnnbrasil.com.br/business/2020/02/...,Ibovespa tem nova queda com mercado ainda preo...,Preocupações com a propagação do novo coronaví...,CNN,preocupações propagação novo coronavírus poten...,preocupações com a propagação do novo coronaví...
1613,https://www.cnnbrasil.com.br/internacional/202...,Japonesa testa positivo pela segunda vez para ...,TÓQUIO - Uma guia de ônibus turístico no Japão...,CNN,tóquio guia ônibus turístico japão apresentou ...,tóquio uma guia de ônibus turístico no japão a...
1614,https://www.cnnbrasil.com.br/saude/2020/02/26/...,Primeiro brasileiro com coronavírus tem sintom...,O primeiro brasileiro com diagnóstico confir...,CNN,primeiro brasileiro diagnóstico confirmado cor...,o primeiro brasileiro com diagnóstico confirma...


#### Tokenizer

In [ ]:
TOKENIZER_NUM_WORDS = 2000

tokenizer = Tokenizer(num_words=TOKENIZER_NUM_WORDS)
tokenizer.fit_on_texts(dataset.processed_text)

### Embedding, convoluções e LSTM

Resolva novamente a segunda questão da 3ª lista usando pelo menos duas
arquiteturas de redes neurais que utilizem camadas *Embedding*, convolucionais e
*LSTM*. Compare com os resultados obtidos anteriormente nas lista 3 e 5.

#### Gerando as sequências

In [ ]:
sequences = tokenizer.texts_to_sequences(dataset.processed_text)
sequences = pad_sequences(sequences)

In [ ]:
INPUT_DIMENSION = TOKENIZER_NUM_WORDS + 1
INPUT_LENGTH = 500
CONV_FILTERS = 32
CONV_KERNEL_SIZE = 3
LSTM_UNITS = 100
DENSE_UNITS = 16

In [ ]:
lstm_model = Sequential([
    Embedding(input_dim=INPUT_DIMENSION, output_dim=300),
    Conv1D(filters=CONV_FILTERS, kernel_size=CONV_KERNEL_SIZE),
    LSTM(units=LSTM_UNITS),
    Dense(units=DENSE_UNITS),
    Dense(units=1, activation='sigmoid')
])

print(lstm_model.summary())

In [ ]:
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.25

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(x=sequences, 
               y=dataset.label.apply(lambda label: 0 if label == 'BBC' else 1),
               batch_size=BATCH_SIZE,
               validation_split=VALIDATION_SPLIT,
               epochs=20)

# Geração de Texto

2. Usando sua base de textos:

* Treine uma rede __LSTM__ para gerar texto, que receba uma ou mais palavras de uma frase como entrada. O treinamento deve ser realizado considerando um conjunto supervisionado que gera a próxima palavra de uma sequência de tamanho 4, usando subsequências de sua base.

* Após o treinamento, exiba pelo menos 5 exemplos de textos dados de entrada, e do texto gerado em seguida pela rede treinada. Para cada exemplo, gere pelo menos 10 palavras consecutivamente.

* Faça o mesmo usando Cadeias de Markov com bi-grams (usando apenas 1 palavra para tentar prever a seguinte). Compare os resultados com os da __LSTM__.



Começamos tokenizando o texto, mapeando cada palavra $w_i$ do nosso vocabulário a um inteiro $i$:

In [11]:
texts = dataset.processed_text_[:50] # somente os 50 primeiros artigos

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

vocabulary_size = len(tokenizer.word_index) + 1

## Método I - Rede Neural LSTM 

In [12]:
sequences = tokenizer.texts_to_sequences(texts)
sequences = pad_sequences(sequences, padding='post')

In [13]:
def generate_subsequences(sequence):
    seq, tar = [], []
    win = [0, 0, 0, sequence[0]]

    for i in sequence[1:]:
        seq.append(win.copy()); tar.append(i)
        for j in range(1, 4):
            win[j-1] = win[j]
        win[3] = i

    return seq, tar

In [14]:
train_sequences, train_targets = [], []

for s in sequences:
    seq, tar = generate_subsequences(s)
    train_sequences.append(seq)
    train_targets.append(tar)

train_sequences = numpy.array([s for train_seq in train_sequences for s in train_seq])
train_targets = to_categorical([t for train_tar in train_targets for t in train_tar]) 

In [15]:
train_sequences.shape

(151050, 4)

In [16]:
train_targets.shape

(151050, 7797)

In [17]:
INPUT_LENGTH = 4
LSTM_UNITS = 100

lstm_model = Sequential([
    Embedding(input_dim=vocabulary_size, output_dim=200, input_length=INPUT_LENGTH),
    LSTM(units=LSTM_UNITS, return_sequences=True),
    LSTM(units=LSTM_UNITS),
    Dense(units=LSTM_UNITS, activation='relu'),
    Dense(units=vocabulary_size, activation='softmax')
])

print(lstm_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 200)            1559400   
_________________________________________________________________
lstm (LSTM)                  (None, 4, 100)            120400    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 7797)              787497    
Total params: 2,557,797
Trainable params: 2,557,797
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
BATCH_SIZE = 64

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

lstm_model.fit(
    x=train_sequences, 
    y=train_targets,
    batch_size=BATCH_SIZE,
    epochs=1,
    verbose=1
)

2361/2361 [==============================] - 106s 45ms/step - loss: 2.5739 - accuracy: 0.6577


In [19]:
def generate_sentence(seed, n):
    words = seed.split(' ')
    words = [tokenizer.word_index[w] for w in words]
    predicted_words = []
    
    for _ in range(n):
        predict = numpy.argmax(lstm_model.predict([words])[0])
        predict_word = tokenizer.index_word[predict]
        predicted_words.append(predict_word)
        for i in range(1, 3):
            words[i] = words[i+1]
        words[3] = predict
    
    return seed + ' ' + ' '.join(predicted_words)

Exemplo de sentença gerada:

In [21]:
generate_sentence(seed='a pandemia do coronavírus', n=15)

'a pandemia do coronavírus de que de que de que de que de que de que de que de'

Salvando o modelo e o tokenizador:

In [ ]:
# lstm_model.save('./model.h5')
# pickle.dump(tokenizer, open('./tokenizer.pkl', 'wb'))

Carregando o modelo já treinado:

In [33]:
lstm_model = load_model('./text_generation/model.h5')

In [35]:
generate_sentence('presidente jair bolsonaro disse', n=10)

'presidente jair bolsonaro disse que faleceu com a mulher afinidade a decisão do instituto'

In [37]:
generate_sentence('a pandemia do coronavírus', n=10)

'a pandemia do coronavírus começou a aliviar a partir do número oficial o que'

In [39]:
generate_sentence('o governo do brasil', n=10)

'o governo do brasil realizado à monarca de anos centenas de cuidados à comunidade'

In [40]:
generate_sentence('a criação da vacina', n=10)

'a criação da vacina em que o novo coronavírus cai apenas entre entre e'

## Método II - Cadeias de Markov

In [27]:
transition_matrix = numpy.zeros(shape=(vocabulary_size, vocabulary_size))

sequences = tokenizer.texts_to_sequences(texts)

for seq in sequences:
    for i in range(len(seq)):
        if i < len(seq) - 1:
            transition_matrix[seq[i]][seq[i+1]] += 1

In [30]:
def build_sentece(initial_word, n=20):
    initial_token = tokenizer.word_index[initial_word]
    result = [initial_word]
    
    for _ in range(n):
        probs = transition_matrix[initial_token] / transition_matrix[initial_token].sum()
        next_token = numpy.random.choice(vocabulary_size, 1, p=probs)[0]
        result.append(tokenizer.index_word[next_token])
        initial_token = next_token
        
    return ' '.join(result)

In [31]:
for word in ['presidente', 'vírus', 'pandemia', 'brasil', 'vacina']:
    print('Palavra de entrada: %s' % word)
    print('Sentença gerada: " %s "\n' % build_sentece(initial_word=word, n=10))

Palavra de entrada: presidente
Sentença gerada: " presidente jair bolsonaro brincou com seu território brasileiro nega a pandemia "

Palavra de entrada: vírus
Sentença gerada: " vírus respiratórios fatais o modo muito mais pobres do país entre "

Palavra de entrada: pandemia
Sentença gerada: " pandemia de autossuficiência alimentar se for interrompido a pele das às "

Palavra de entrada: brasil
Sentença gerada: " brasil o piauiense passou a ver com o conselho nacional fica "

Palavra de entrada: vacina
Sentença gerada: " vacina nacional de casos como também porque não considera nessa época "

